# Data

s3://thedataincubator-course/mrdata/simple/

In [2]:
!wget https://s3.amazonaws.com/thedataincubator-course/mrdata/simple/part-00026.xml.bz2  

--2016-04-05 14:16:51--  https://s3.amazonaws.com/thedataincubator-course/mrdata/simple/part-00026.xml.bz2
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.11.131
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.11.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2897309 (2.8M) [application/xml]
Saving to: ‘part-00026.xml.bz2’

100%[======================================>] 2,897,309   --.-K/s   in 0.1s    

2016-04-05 14:16:51 (21.9 MB/s) - ‘part-00026.xml.bz2’ saved [2897309/2897309]



In [5]:
!ls

MR-Wikipedia.ipynb  part-00026.xml.bz2


## Overiding steps method

https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs

### Running job hadoop (multiple cores)

python job_file.py -r hadoop s3n://thedataincubator-course/mrdata/simple/  

### Running job local

python job_file.py -r local s3n://thedataincubator-course/mrdata/simple/ 

Here are some helpful articles on how mrjob works and how to pass parameters to
your script:
  - [How mrjob is
    run](https://pythonhosted.org/mrjob/guides/concepts.html#how-your-program-is-run)
  - [Adding passthrough
  options](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.add_passthrough_option)
  - [An example of someone solving similar
  problems](http://arunxjacob.blogspot.com/2013/11/hadoop-streaming-with-mrjob.html)



## top100_words_simple_plain

In [331]:
%%writefile top100_words_simple_plain.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
WORD_RE = re.compile(r"\w+")

class MRSimplePlain(MRJob):
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            self.increment_counter("Word Counter", "Total words", amount=1)
            yield (word.lower(), 1)
    def combiner(self, word, counts):
        yield (word, sum(counts))
    # input is: key, value
    def reducer(self, word, counts):
        self.increment_counter("Word Counter", "Unique words", amount=1)
        yield (word, sum(counts))
    def mapper2_init(self):
        self.heap = []
        self.nlargest=[]
    def mapper2(self,word,counts):
        #for pair in pairs :
        heapq.heappush(self.heap, (word,counts))
#        self.n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
    def mapper2_final(self):
        yield (None,self.heap)
#        yield (None,self.n_largest)
    
    def reducer2_init(self):
        self.new_heap=[]
        self.final=[]
    
    def reducer2(self,_,heap):
        count=0
        for h in heap:
            self.new_heap=heapq.chain(self.new_heap,h)
        self.final=heapq.nlargest(100,self.new_heap,key=lambda e:e[1])
    def reducer2_final(self):
        print [(i[0], i[1]) for i in self.final]
        yield (None,self.final)
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(mapper_init=self.mapper2_init, mapper=self.mapper2
                 ,mapper_final=self.mapper2_final
            ,reducer_init=self.reducer2_init
                 , reducer=self.reducer2,reducer_final=self.reducer2_final)
            ]
if __name__ == '__main__':
    MRSimplePlain.run()

Overwriting top100_words_simple_plain.py


In [332]:
%reload_ext autoreload
%autoreload 4
from top100_words_simple_plain import MRSimplePlain
filename = 'sample.xml'
##filename = 'part-00026.xml'

import time
start_time = time.time()

mr_job = MRSimplePlain(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



[('the', 23), ('id', 18), ('b', 13), ('category', 12), ('marathon', 12), ('of', 11), ('it', 10), ('new', 10), ('text', 9), ('title', 9), ('york', 9), ('in', 8), ('a', 7), ('page', 7), ('city', 6), ('contributor', 6), ('format', 6), ('gt', 6), ('lt', 6), ('model', 6), ('ns', 6), ('ref', 6), ('revision', 6), ('sha1', 6), ('timestamp', 6), ('username', 6), ('was', 6), ('2013', 5), ('bism', 5), ('by', 5), ('http', 5), ('ing', 5), ('is', 5), ('suffolk', 5), ('to', 5), ('11', 4), ('and', 4), ('comment', 4), ('november', 4), ('parentid', 4), ('quot', 4), ('sports', 4), ('www', 4), ('2012', 3), ('bridge', 3), ('cancelled', 3), ('com', 3), ('for', 3), ('from', 3), ('on', 3), ('people', 3), ('preserve', 3), ('religion', 3), ('space', 3), ('sportspeople', 3), ('verrazano', 3), ('wiki', 3), ('wikitext', 3), ('x', 3), ('xml', 3), ('0', 2), ('2010', 2), ('at', 2), ('because', 2), ('been', 2), ('cite', 2), ('english', 2), ('events', 2), ('has', 2), ('he', 2), ('hotcat', 2), ('htm', 2), ('interpretati

In [23]:
! python top100_words_simple_plain.py -r hadoop s3n://thedataincubator-course/mrdata/simple/

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/top100_words_simple_plain.vagrant.20160407.221138.022974
writing wrapper script to /tmp/top100_words_simple_plain.vagrant.20160407.221138.022974/setup-wrapper.sh
Using Hadoop version 2.5.0
Copying local files into hdfs:///user/vagrant/tmp/mrjob/top100_words_simple_plain.vagrant.20160407.221138.022974/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.5.0-cdh5.3.5.jar] /tmp/streamjob1267738755138315539.jar tmpDir=null
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Permissions on staging directory /tmp/hadoop-yarn/staging/vagrant/.staging are

## top100_words_simple_text

In [335]:
%%writefile top100_words_simple_text.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
WORD_RE = re.compile(r"\w+")


class MRSimpleText(MRJob):
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0
    
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            yield (self.counter,self.page)

        


    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[-1]
            texts = texts.findall(".//text")
            text_string = etree.tostring(texts[0])
            yield (None,text_string)
        
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            self.increment_counter("Word Counter", "Total words", amount=1)
            yield (word.lower(), 1)
    # input is: key, value
    def reducer(self, word, counts):
        self.increment_counter("Word Counter", "Unique words", amount=1)
        yield (word, sum(counts))
 
    def mapper2_init(self):
        self.heap = []
        self.nlargest=[]
    def mapper2(self,word,counts):
        #for pair in pairs :
        heapq.heappush(self.heap, (word,counts))
#        self.n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
    def mapper2_final(self):
        n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
        yield (None,n_largest)
#        yield (None,self.n_largest)
    
    def reducer2_init(self):
        self.new_heap=[]
        self.final=[]
    
    def reducer2(self,_,heap):
        count=0
        for h in heap:
            self.new_heap=heapq.chain(self.new_heap,h)
        self.final=heapq.nlargest(100,self.new_heap,key=lambda e:e[1])
    def reducer2_final(self):
        print [(i[0], i[1]) for i in self.final]
        yield (None,self.final)
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper=self.mapper,
            #       combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(mapper_init=self.mapper2_init, mapper=self.mapper2
                 ,mapper_final=self.mapper2_final
            ,reducer_init=self.reducer2_init
                 , reducer=self.reducer2,reducer_final=self.reducer2_final)
            ]
if __name__ == '__main__':
    MRSimpleText.run()         

Overwriting top100_words_simple_text.py


In [336]:
%reload_ext autoreload
%autoreload 4
from top100_words_simple_text import MRSimpleText
##filename = 'sample.xml'
filename = 'part-00026.xml'

import time
start_time = time.time()

mr_job = MRSimpleText(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



[('the', 45047), ('lt', 32247), ('gt', 32218), ('of', 28145), ('in', 20342), ('ref', 18862), ('and', 16703), ('category', 16079), ('text', 15050), ('a', 14211), ('to', 11198), ('is', 10629), ('was', 9724), ('http', 8973), ('amp', 8074), ('space', 7430), ('s', 7354), ('preserve', 7327), ('xml', 7298), ('name', 7192), ('on', 6731), ('it', 6656), ('www', 6284), ('for', 5605), ('he', 5439), ('title', 5266), ('from', 5225), ('com', 5211), ('2013', 4667), ('by', 4465), ('as', 4323), ('br', 4049), ('url', 4006), ('at', 3853), ('2014', 3826), ('people', 3735), ('align', 3683), ('that', 3593), ('1', 3585), ('8211', 3578), ('an', 3562), ('with', 3421), ('accessdate', 3385), ('publisher', 3379), ('cite', 3371), ('center', 3367), ('this', 3334), ('date', 3137), ('american', 3041), ('stub', 2961), ('united', 2872), ('be', 2860), ('2', 2811), ('new', 2772), ('other', 2750), ('talk', 2748), ('references', 2745), ('web', 2690), ('his', 2669), ('user', 2614), ('states', 2601), ('first', 2562), ('are', 

In [231]:
! python top100_words_simple_text.py -r hadoop s3n://thedataincubator-course/mrdata/simple/

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/top100_words_simple_text.vagrant.20160410.031026.176339
writing wrapper script to /tmp/top100_words_simple_text.vagrant.20160410.031026.176339/setup-wrapper.sh
Using Hadoop version 2.5.0
Copying local files into hdfs:///user/vagrant/tmp/mrjob/top100_words_simple_text.vagrant.20160410.031026.176339/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.5.0-cdh5.3.5.jar] /tmp/streamjob2566697572191504525.jar tmpDir=null
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Permissions on staging directory /tmp/hadoop-yarn/staging/vagrant/.staging are in

# top100_words_simple_no_metadata

In [503]:
%%writefile top100_words_simple_no_metadata.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
import mwparserfromhell
import unicodedata


WORD_RE = re.compile(r"\w+")


class MRSimpleNoMeta(MRJob):
    
    #### Step 1 : Consolidate pages ####
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0 
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            yield (self.counter,self.page)
    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[-1]
            texts = texts.findall(".//text")
            text_string = etree.tostring(texts[0])
            wikicode = mwparserfromhell.parse(text_string)
            text = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())
            text = text.encode('utf-8') 
            text = re.sub(r'http', ' ', text, flags=re.IGNORECASE)
            text = re.sub(r'www', ' ', text, flags=re.IGNORECASE)
            yield (None,text)
    
    
    
    ##### Step 2 : Count words   #########
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            self.increment_counter("Word Counter", "Total words", amount=1)
            yield (word.lower(), 1)
    # input is: key, value
    def reducer(self, word, counts):
        self.increment_counter("Word Counter", "Unique words", amount=1)
        yield (word, sum(counts))
 
    #### Step 3 : Yield top 100 words ####
    def mapper2_init(self):
        self.heap = []
        self.nlargest=[]
    def mapper2(self,word,counts):
        #for pair in pairs :
        heapq.heappush(self.heap, (word,counts))
#        self.n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
    def mapper2_final(self):
        n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
        yield (None,n_largest)
#        yield (None,self.n_largest)
    
    def reducer2_init(self):
        self.new_heap=[]
        self.final=[]
    
    def reducer2(self,_,heap):
        count=0
        for h in heap:
            self.new_heap=heapq.chain(self.new_heap,h)
        self.final=heapq.nlargest(100,self.new_heap,key=lambda e:e[1])
    def reducer2_final(self):
        print [(i[0], i[1]) for i in self.final]
        yield (None,self.final)
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper=self.mapper,
            #       combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(mapper_init=self.mapper2_init, mapper=self.mapper2
                 ,mapper_final=self.mapper2_final
            ,reducer_init=self.reducer2_init
                 , reducer=self.reducer2,reducer_final=self.reducer2_final)
            ]
if __name__ == '__main__':
    MRSimpleNoMeta.run()         

Overwriting top100_words_simple_no_metadata.py


In [504]:
%reload_ext autoreload
%autoreload 4
from top100_words_simple_no_metadata import MRSimpleNoMeta
filename = 'sample.xml'
##filename = 'part-00026.xml'
##filename = 'simple'

import time
start_time = time.time()

mr_job = MRSimpleNoMeta(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



[('the', 23), ('b', 12), ('marathon', 11), ('it', 10), ('of', 10), ('new', 9), ('category', 8), ('in', 8), ('york', 8), ('a', 7), ('ref', 6), ('text', 6), ('was', 6), ('city', 5), ('ing', 5), ('is', 5), ('to', 5), ('and', 4), ('bism', 4), ('by', 4), ('november', 4), ('2012', 3), ('bridge', 3), ('cancelled', 3), ('com', 3), ('for', 3), ('on', 3), ('preserve', 3), ('religion', 3), ('space', 3), ('sports', 3), ('title', 3), ('verrazano', 3), ('xml', 3), ('2010', 2), ('2013', 2), ('at', 2), ('because', 2), ('been', 2), ('cite', 2), ('has', 2), ('he', 2), ('htm', 2), ('interpretation', 2), ('islam', 2), ('narrows', 2), ('nyc', 2), ('nycmarathon', 2), ('official', 2), ('org', 2), ('other', 2), ('people', 2), ('persia', 2), ('publisher', 2), ('race', 2), ('religions', 2), ('runners', 2), ('stub', 2), ('suffolk', 2), ('thumb', 2), ('url', 2), ('used', 2), ('web', 2), ('11', 1), ('1844', 1), ('1970', 1), ('19th', 1), ('2', 1), ('219', 1), ('255px', 1), ('26', 1), ('304', 1), ('50', 1), ('58658'

In [449]:
%%writefile top100_words_simple_no_metadata.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
import sys
##sys.path.append('/home/vagrant/mwparserfromhell')
##hdfs:///user/vagrant/tmp/mrjob/top100_words_simple_no_metadata.vagrant.20160409.193526.490917/files/
sys.path.append('/opt/conda/lib/python2.7/site-packages/mwparserfromhell-0.5.dev0-py2.7-linux-x86_64.egg')
import mwparserfromhell



WORD_RE = re.compile(r"\w+")


class MRSimpleTextNoMeta(MRJob):
    
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0
    
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            yield (self.counter,self.page)
        


    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[-1]
            texts = texts.findall(".//text")
            texts = etree.tostring(texts[0])
            wikicode = mwparserfromhell.parse(texts)
            texts = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())
            texts = str(texts).decode("ISO-8859-1").encode("utf-8")
            
            ##wikicode = mwparserfromhell.parse(texts)
            ##texts = wikicode.filter_text()
            ##for text in texts:   
              ##  if '://' not in str(text):
                ##    print text
                 ##   yield (None,str(text))
            yield(None,texts)

    
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            ##if ' ' in word:
                ##word=' '.join(word.split())
            self.increment_counter("Word Counter", "Total Words", amount=1)
            yield (word.lower(), 1)
    # input is: key, value
    def reducer(self, word, counts):
        self.increment_counter("Word Counter", "Unique Words", amount=1)
        yield (word, sum(counts))

 
    def mapper2_init(self):
        self.heap = []
        self.nlargest=[]
    def mapper2(self,word,counts):
        #for pair in pairs :
        heapq.heappush(self.heap, (word,counts))
#        self.n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
    def mapper2_final(self):
        n_largest = heapq.nlargest(100,self.heap,key=lambda e:e[1])
        yield (None,n_largest)
#        yield (None,self.n_largest)
    
    def reducer2_init(self):
        self.new_heap=[]
        self.final=[]
    
    def reducer2(self,_,heap):
        count=0
        for h in heap:
            self.new_heap=heapq.chain(self.new_heap,h)
        self.final=heapq.nlargest(100,self.new_heap,key=lambda e:e[1])
    def reducer2_final(self):
        print [(i[0], i[1]) for i in self.final]
        yield (None,self.final)
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper=self.mapper,
            #       combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(mapper_init=self.mapper2_init, mapper=self.mapper2
                 ,mapper_final=self.mapper2_final
            ,reducer_init=self.reducer2_init
                 , reducer=self.reducer2,reducer_final=self.reducer2_final)
            ]
if __name__ == '__main__':
    MRSimpleTextNoMeta.run()         

Overwriting top100_words_simple_no_metadata.py


In [279]:
! python top100_words_simple_no_metadata.py -r hadoop s3n://thedataincubator-course/mrdata/simple/

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/top100_words_simple_no_metadata.vagrant.20160410.193553.908170
writing wrapper script to /tmp/top100_words_simple_no_metadata.vagrant.20160410.193553.908170/setup-wrapper.sh
Using Hadoop version 2.5.0
Copying local files into hdfs:///user/vagrant/tmp/mrjob/top100_words_simple_no_metadata.vagrant.20160410.193553.908170/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.5.0-cdh5.3.5.jar] /tmp/streamjob8577741307769624639.jar tmpDir=null
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Permissions on staging directory /tmp/hadoop-yarn/staging/va

# wikipedia_entropy

https://en.wikipedia.org/wiki/Entropy_(information_theory) 

The data you need is available at:
    - https://s3.amazonaws.com/thedataincubator-course/mrdata/simple/part-000\*
    - https://s3.amazonaws.com/thedataincubator-course/mrdata/thai/part-000\*



http://www.johndcook.com/blog/2013/08/17/calculating-entropy/

# wikipedia_entropy

In [679]:
%%writefile wikipedia_entropy.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
import mwparserfromhell
import unicodedata
import math

WORD_RE = re.compile(r"\w+")
           
           
           
class MREntropyCalc(MRJob):
    
    #### Step 1 : Consolidate pages ####
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0 
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            ##if self.counter > 0 and self.counter < 100:
            yield (self.counter,self.page)     
            
            
    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[-1]
            texts = texts.findall(".//text")
            text_string = etree.tostring(texts[0])
            wikicode = mwparserfromhell.parse(text_string)
            text = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())
            text = text.encode('utf-8') 
            text = re.sub(r'http', ' ', text, flags=re.IGNORECASE)
            text = re.sub(r'www', ' ', text, flags=re.IGNORECASE)
            yield (None,text)
    
    

    ##### Step 2 : Count characters   #########
    def mapper_init_charcount(self):
        self.unigram = None
        self.twogram = None
        self.threegram = None
        
      
    def mapper_charcount(self, _, line):
        for char in line:
            if self.twogram:
                self.threegram = self.twogram + char.encode('utf-8')
            if self.unigram:
                self.twogram = self.unigram +  char.encode('utf-8')
            self.unigram = char.encode('utf-8')      
            if self.unigram:
                self.increment_counter("Char Counter", "Total unigram", amount=1)
                self.increment_counter("Char Counter", "Total", amount=1)
                yield (self.unigram,1)
            if self.twogram:
                self.increment_counter("Char Counter", "Total twogram", amount=1)
                self.increment_counter("Char Counter", "Total", amount=1)
                yield (self.twogram,1)

            if self.threegram:
                self.increment_counter("Char Counter", "Total threegram", amount=1)
                self.increment_counter("Char Counter", "Total", amount=1)
                yield (self.threegram,1)
    # input is: key, value
    def reducer_init_charcount(self):
        self.total_uni_N = 0
        self.total_uni_ngram_sum = 0
        self.total_two_N = 0
        self.total_two_ngram_sum = 0
        self.total_three_N = 0
        self.total_three_ngram_sum = 0  

           
    
    def reducer_charcount(self, char,counts):
        self.increment_counter("Char Counter", "Unique characters", amount=1)
        self.n = sum(counts)
        self.ngram_sum = self.n*math.log(self.n,2)
        
        if len(char) == 1:
            self.increment_counter("Char Counter", "Unique unigrams", amount=1)
            ##yield(None,(char,self.n,self.ngram_sum))
            self.total_uni_N += self.n
            self.total_uni_ngram_sum += self.ngram_sum
            self.uni_entropy= math.log(self.total_uni_N,2)- self.total_uni_ngram_sum/self.total_uni_N
        elif len(char) == 2:
            self.increment_counter("Char Counter", "Unique twograms", amount=1)
            self.total_two_N += self.n
            self.total_two_ngram_sum += self.ngram_sum
            self.two_entropy= (math.log(self.total_two_N,2)- self.total_two_ngram_sum/self.total_two_N)/2
        else:
            self.increment_counter("Char Counter", "Unique threegrams", amount=1)
            self.total_three_N += self.n
            self.total_three_ngram_sum += self.ngram_sum
            self.three_entropy= (math.log(self.total_three_N,2)- self.total_three_ngram_sum/self.total_three_N)/3
        
  



    def reducer_charcount_final(self):
        
        print self.uni_entropy
        print self.two_entropy
        print self.three_entropy
    
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper_init=self.mapper_init_charcount,
                   mapper=self.mapper_charcount,
            #       combiner=self.combiner,
                   reducer_init=self.reducer_init_charcount,
                   reducer=self.reducer_charcount,
                   reducer_final=self.reducer_charcount_final)
            ]
if __name__ == '__main__':
    MREntropyCalc.run()         

Overwriting wikipedia_entropy.py


In [738]:
%reload_ext autoreload
%autoreload 4
from wikipedia_entropy import MREntropyCalc
##filename = 'sample.xml'
##filename = 'part-00026.xml'
##filename = 'simple'
filename = 'thaipart-00027.xml'

import time
start_time = time.time()

mr_job = MREntropyCalc(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



5.08946972558
4.55384200601
4.05304806367
[{'Word Counter': {'Pages Reviewed': 12402}}, {'Char Counter': {'Total twogram': 6044723, 'Unique unigrams': 92, 'Unique characters': 84544, 'Unique threegrams': 77798, 'Total unigram': 6044724, 'Total': 18134169, 'Total threegram': 6044722, 'Unique twograms': 6654}}]
--- 11.632469217 min ---


In [627]:
! python wikipedia_entropy.py -r hadoop sample.xml

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/wikipedia_entropy.vagrant.20160411.233644.093260
writing wrapper script to /tmp/wikipedia_entropy.vagrant.20160411.233644.093260/setup-wrapper.sh
Using Hadoop version 2.5.0
Copying local files into hdfs:///user/vagrant/tmp/mrjob/wikipedia_entropy.vagrant.20160411.233644.093260/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.5.0-cdh5.3.5.jar] /tmp/streamjob9021901117161648229.jar tmpDir=null
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Permissions on staging directory /tmp/hadoop-yarn/staging/vagrant/.staging are incorrect: rwxrwxrwt. F

 # Top NGrams

In [739]:
%%writefile top_ngrams_entropy.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
import mwparserfromhell
import unicodedata
import math

WORD_RE = re.compile(r"\w+")


class MRTopNGrams(MRJob):
    
    #### Step 1 : Consolidate pages ####
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0 
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            if self.counter > 0 and self.counter < 104:
                yield (self.counter,self.page)     
            
            
    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[-1]
            texts = texts.findall(".//text")
            text_string = etree.tostring(texts[0])
            wikicode = mwparserfromhell.parse(text_string)
            text = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())
            text = text.encode('utf-8') 
            text = re.sub(r'http', ' ', text, flags=re.IGNORECASE)
            text = re.sub(r'www', ' ', text, flags=re.IGNORECASE)
            yield (None,text)
    
    

    ##### Step 2 : Count characters   #########
    def mapper_init_charcount(self):
        None
      
    def mapper_charcount(self, _, line):
        char_count = len(line)
        self.increment_counter("Word Counter", "Lines reviewed", amount=1)
   
        yield(None, char_count)
    # input is: key, value
    
    
    def reducer_charcount(self, _,char_count):
        n = sum(char_count)
        print n
        uni_nsample = 6044724
        uni_ksample = 92
        two_ksample = 6654
        three_ksample = 77798
        
        def prob_calc(k,n):
            k_sum = 0 
            for i in range (1,k+1):
                k_sum += 1.0/i             
            return 1.0/k_sum
        
        def entropy_calc(ksample,nsample,char,nactual):
            ngram_sum = 0
            max_count = prob_calc(ksample,nsample)*nactual
            for i in range(1,ksample+1):
                ncount =  max_count*1.0/i  
                ngram_sum += ncount*math.log(ncount,2)
            entropy = math.log(nactual,2)- (ngram_sum/nactual)
            return entropy/char
        
        one = entropy_calc(uni_ksample, uni_nsample, 1,n )
        two = entropy_calc(two_ksample, uni_nsample-1, 2,n )
        three = entropy_calc(three_ksample, uni_nsample-2, 3,n )
        yield(None, (one,two,three))
        print one, two, three

        
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper_init=self.mapper_init_charcount,
                   mapper=self.mapper_charcount,
            #       combiner=self.combiner,
                   reducer=self.reducer_charcount)
            ]
if __name__ == '__main__':
    MRTopNGrams.run()         

Overwriting top_ngrams_entropy.py


In [741]:
%reload_ext autoreload
%autoreload 4
from top_ngrams_entropy import MRTopNGrams
##filename = 'sample.xml'
##filename = 'part-00026.xml'
##filename = 'simple'
filename = 'wikithai'
import time
start_time = time.time()

mr_job = MRTopNGrams(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



2525795
5.22757306473 4.58886758288 3.76142266511
[{'Word Counter': {'Pages Reviewed': 3914}}, {'Word Counter': {'Lines reviewed': 3914}}]
--- 2.99097809792 min ---


In [728]:
import math
uni_nsample = 8594385
uni_ksample = 92
two_ksample = 6424
three_ksample = 72879
        
def prob_calc(k,n):
    k_sum = 0 
    for i in range (1,k+1):
        k_sum += 1.0/i 
    return 1.0/k_sum
        
print prob_calc(uni_ksample,uni_nsample)

def entropy_calc(ksample,nsample,char,nactual):
    ngram_sum = 0
    max_count = prob_calc(ksample,nsample)*nactual
    print max_count
    for i in range(1,ksample+1):
        ncount =  max_count*1.0/i  
        ngram_sum += ncount*math.log(ncount,2)
    entropy = math.log(nactual,2)- (ngram_sum/nactual)
    return entropy/char
entropy_calc(uni_ksample,uni_nsample,1,8594385)

0.195908291588
1683711.2826


5.227573064725462

In [181]:
! python most_common_char_en.py -r hadoop sample.xml

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/most_common_char_en.vagrant.20160410.003026.711509
writing wrapper script to /tmp/most_common_char_en.vagrant.20160410.003026.711509/setup-wrapper.sh
Using Hadoop version 2.5.0
Copying local files into hdfs:///user/vagrant/tmp/mrjob/most_common_char_en.vagrant.20160410.003026.711509/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.5.0-cdh5.3.5.jar] /tmp/streamjob4340401832827294728.jar tmpDir=null
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Connecting to ResourceManager at alejandro-j-rojas/127.0.1.1:8032
HADOOP: Total input paths to process : 1
HADOOP: number of splits:2
HADOOP: Submitting tokens for job: j

In [102]:
! cat sample.xml

<page>
    <title>New York City Marathon</title>
    <ns>0</ns>
    <id>423724</id>
    <revision>
      <id>4615420</id>
      <parentid>4615295</parentid>
      <timestamp>2013-11-10T08:18:23Z</timestamp>
      <contributor>
        <username>Osiris</username>
        <id>209999</id>
      </contributor>
      <minor />
      <comment>added [[Category:Sports events]] using [[Help:Gadget-HotCat|HotCat]]</comment>
      <text xml:space="preserve">[[Image:New York marathon Verrazano bridge.jpg|thumb|255px|Marathon runners on Verrazano-Narrows Bridge.]]
The '''New York City Marathon''' (called the '''ING New York City Marathon''' because it is [[sponsor]]ed by [[financial]] company [[ING Group]]) is a yearly [[marathon]] that goes through all five [[borough]]s of [[New York City]]. The distance of the race is 26.219 miles. It starts at the [[Verrazano-Narrows Bridge]] and ends at [[Columbus Circle]]. It takes place on the first Sunday of November.&lt;ref&gt;http://gonyc.about.com/od/plan

In [ ]:
       wikicode = mwparserfromhell.parse(text)
        meta_text = wikicode.filter_templates()
        text =  meta_text.filter_text()

In [112]:
text_test = '<comment>Created page with &quot;[[Category:English sportspeople by location of origin|Suffolk]] [[Category:People from Suffolk]]&quot;</comment><text xml:space="preserve">[[Category:English sportspeople by location of origin|Suffolk]][[Category:People from Suffolk]]</text>'

In [113]:
wikicode = mwparserfromhell.parse(text_test)

In [114]:
wikicode

u'<comment>Created page with &quot;[[Category:English sportspeople by location of origin|Suffolk]] [[Category:People from Suffolk]]&quot;</comment><text xml:space="preserve">[[Category:English sportspeople by location of origin|Suffolk]][[Category:People from Suffolk]]</text>'

In [117]:
meta_text = wikicode.filter_text()

In [118]:
meta_text

[u'comment',
 u'Created page with ',
 u'Category:English sportspeople by location of origin',
 u'Suffolk',
 u' ',
 u'Category:People from Suffolk',
 u'comment',
 u'text',
 u'xml:space',
 u'preserve',
 u'Category:English sportspeople by location of origin',
 u'Suffolk',
 u'Category:People from Suffolk',
 u'text']

In [128]:
str(meta_text[0])

'comment'

In [120]:
templates

[]

In [350]:
test_text = '{commons category}}*{{official website|http://www.nycmarathon.org/}}*[http://www.crowdrise.com/ing-nyc-marathon-2013 Official ING NYC Marathon Fundraising Site]'

In [381]:
wikicode = mwparserfromhell.parse(test_text)

In [382]:
meta_text = wikicode.filter_text()

In [383]:
meta_text

[u'{commons category}}*',
 u'official website',
 u'http://www.nycmarathon.org/',
 u'*',
 u'http://www.crowdrise.com/ing-nyc-marathon-2013',
 u'Official ING NYC Marathon Fundraising Site']

In [384]:
textst = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())

In [385]:
textst

u'{commons category}}* official website http://www.nycmarathon.org/ * http://www.crowdrise.com/ing-nyc-marathon-2013 Official ING NYC Marathon Fundraising Site'

In [386]:
textst = str(textst).decode("ISO-8859-1").encode("utf-8")

In [390]:
textst

'{commons category}}* official website http://www.nycmarathon.org/ * http://www.crowdrise.com/ing-nyc-marathon-2013 Official ING NYC Marathon Fundraising Site'

In [391]:
textst = re.sub(r'http', ' ', textst, flags=re.IGNORECASE)
 

In [393]:
textst = re.sub(r'www', ' ', textst, flags=re.IGNORECASE)

In [394]:
textst

'{commons category}}* official website  :// .nycmarathon.org/ *  :// .crowdrise.com/ing-nyc-marathon-2013 Official ING NYC Marathon Fundraising Site'

In [ ]:
wikicode = mwparserfromhell.parse(text_string)
        texts = [" ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_text())]
        for text in texts:
            text = str(text).decode("ISO-8859-1").encode("utf-8")
            yield (None,text) 

In [142]:
for text in meta_text:
    if ':' not in text:
        print (None,str(text))

(None, '{commons category}}*')
(None, 'official website')
(None, '*')
(None, 'Official ING NYC Marathon Fundraising Site')


In [266]:
%%writefile top100_words_simple_no_metadata.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import heapq
import hashlib
from lxml import etree
import mwparserfromhell

WORD_RE = re.compile(r"\w+")


class MRSimpleTextNoMeta(MRJob):

    def mapper_page_init(self):

        self.page = ""
        self.counter = 0
    
    def mapper_page (self,_,line):
        #print self.counter
        
        if '<page>' in line : 
            self.page = line
        else :
            self.page = self.page +" "+ line
        if '</page>' in line :
            hash_object = hashlib.md5(self.page).hexdigest()
            yield (hash_object,self.page)
        #    yield ("page",self.page)
    
    def reducer_init(self):
        self.counter=0
        def remove_brackets(x):
            #rep="\[\[.*?\]\]"
            rep1="\{\{.*?\}\}"
            rep2="\<.*?\>"
            rep3="\[[.*?\]]"
            a=re.sub(rep1,' ',x)
            a=re.sub(rep2,' ',a)
            a=re.sub(rep3,' ',a)
            return (a) 
        self.remove_brackets=remove_brackets
        
        
    def reducer_page(self, key, pchunk):
        for chunk in pchunk:
            self.counter+=1
            #chunk='<page>'+' '+ chunk
            #if self.counter<5:
                #print chunk
            
            #if self.counter<5:
            #    print 
            #print self.counter
            #print chunk
            parser = etree.XMLParser(recover=True)
            
            
            #if self.counter==1032:
                #chunk='<page> <revision> <text> aaaa </text> </revision> </page>'
                #print chunk
            
            x = etree.fromstring(chunk,parser=parser)
            #print 'xqqqqqqqqqqqqqq'
           
            #print etree.tostring(x)
            if x is not None:
                revs = x.findall(".//revision")
                if len (revs)>0:
                    revs=revs[-1]
                    texts = revs.findall(".//text")
                    if len (texts)>0:
                        text_string = etree.tostring(texts[0])
                        wikicode = mwparserfromhell.parse(text_string)
                        text_string = " ".join(" ".join(fragment.value.split()) for fragment in wikicode.filter_text())
                #print 'XXXXXX'
                        ###print text_string
                        yield (key,text_string)
            #wikicode = mwparserfromhell.parse(text_string)
            #text_string = wikicode.filter_text()
            #text_string=''.join(str(text_string))
            #text_string=self.remove_brackets(text_string)
            
     
    
    def get_words(self, _, line):
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def sum_words(self, word, counts):
        #for word,count in counts:
            yield (word, sum(counts))
            
    def map_init(self):
        self.heap=[]
       
    def map2(self,word,counts):
        heapq.heappush(self.heap,(counts,word))
        
    def map_final(self):
        yield ("batch",self.heap)
    
    def reducer2_init(self):
        self.new_heap=[]
    
    def reducer2(self,_,heap):
        count=0
        for h in heap:
            self.new_heap=heapq.chain(self.new_heap,h)
           
        n_largest=heapq.nlargest(100,self.new_heap)
        #for i in n_largest:
        #    print i[::1]
        q=[(i[1],i[0]) for i in n_largest]
        print q
        #print n_largest
        #print n_largest
        
        yield (None, q)
    
                       
    def steps(self):
        return [MRStep(mapper_init=self.mapper_page_init,
                       mapper=self.mapper_page,
                       reducer_init=self.reducer_init,
                       reducer=self.reducer_page),
                MRStep(mapper=self.get_words,
                       reducer=self.sum_words),
                MRStep(mapper_init=self.map_init,
                       mapper=self.map2,
                       mapper_final=self.map_final,
                       reducer_init=self.reducer2_init,
                       reducer=self.reducer2)]
                       
if __name__ == '__main__':
    #x = re.findall("<page>[\s\S]*?<\/page>",part)
    #len(x)
    MRSimpleTextNoMeta.run()

Overwriting top100_words_simple_no_metadata.py


## Double Links

In [ ]:
%%writefile top_ngrams_entropy.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import heapq
import re
import lxml
from lxml import etree
import mwparserfromhell
import unicodedata
import math

WORD_RE = re.compile(r"\w+")


class DoubleLinks(MRJob):
    
    #### Step 1 : Consolidate pages ####
    def mapper_page_init(self):
        self.page = ""
        self.counter = 0 
    def mapper_page (self,_,line):
        if '<page>' in line : 
            self.page = line
            self.counter += 1
        else:
            self.page = self.page +" "+ line
        
        if '</page>' in line :           
            if self.counter > 0 and self.counter < 104:
                yield (self.counter,self.page)     
            
            
    def reducer_page(self, _, pchunk):
        for chunk in pchunk:
            self.increment_counter("Word Counter", "Pages Reviewed", amount=1)
            texts = etree.fromstring(chunk)
            ##if '<revision>'in texts:
            texts = texts.findall(".//revision")[1]
            texts = texts.findall(".//text")
            text_string = etree.tostring(texts[0])
            wikicode = mwparserfromhell.parse(text_string)
            text = " ".join(" ".join(fragment.value.split())
                      for fragment in wikicode.filter_wikilinks())
            text = text.encode('utf-8') 
            yield (None,text)
    

        
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_page_init,
                   mapper=self.mapper_page,
                   reducer=self.reducer_page
                  ),
            MRStep(mapper_init=self.mapper_init_charcount,
                   mapper=self.mapper_charcount,
            #       combiner=self.combiner,
                   reducer=self.reducer_charcount)
            ]
if __name__ == '__main__':
    MRTopNGrams.run()           

In [ ]:
%reload_ext autoreload
%autoreload 4
from top_ngrams_entropy import MRTopNGrams
##filename = 'sample.xml'
##filename = 'part-00026.xml'
filename = 'simple'
##filename = 'wikithai'
import time
start_time = time.time()

mr_job = MRTopNGrams(args=[filename])
with mr_job.make_runner() as runner:
    runner.run()
    print runner.counters()
    ##for line in runner.stream_output():
        ##print line

print("--- %s min ---" % (((time.time() - start_time))/60)  )  



In [784]:
answer = [(('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'), 0.06825002019931593), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'departments of france'), 0.06816959025546497), (('communes of the vend\\u00e9e department', 'france'), 0.06732691607213888), (('illinois', 'list of cities in illinois'), 0.06709503610594328), (('communes of the allier department', 'france'), 0.0669888479020342), (('communes of the allier department', 'departments of france'), 0.0667632804953407), (('communes of the yonne department', 'france'), 0.0665933570329292), (('communes of the yonne department', 'departments of france'), 0.06649728320993599), (('list of cities in illinois', 'united states'), 0.06642777388943956), (('city', 'list of cities in illinois'), 0.06486918837762555), (('communes of the mayenne department', 'france'), 0.06441469801963563), (('list of cities in indiana', 'united states'), 0.0642825118200217), (('dolj county', 'romania'), 0.061818313540956286), (('communes of the pas-de-calais department', 'france'), 0.06063394551773228), (('communes of the pas-de-calais department', 'departments of france'), 0.060534167840750566), (('iowa', 'list of cities in iowa'), 0.060249012058122484), (('communes of the aisne department', 'france'), 0.06018953539234461), (('communes of the aisne department', 'departments of france'), 0.06013498206908435), (('list of cities in iowa', 'united states'), 0.05996983823372155), (('city', 'list of cities in iowa'), 0.05989148958097098), (('communes of the gironde department', 'france'), 0.059672265545566656), (('communes of the gironde department', 'departments of france'), 0.05954958923801437), (('communes of the sarthe department', 'france'), 0.058845936015815774), (('communes of the sarthe department', 'departments of france'), 0.05873030518050841), (('list of cities in oklahoma', 'united states'), 0.05837812057467899), (('list of cities in oklahoma', 'oklahoma'), 0.05835728724134566), (('communes of the ard\\u00e8che department', 'france'), 0.05825406944175671), (('communes of the ain department', 'france'), 0.05800335494685684), (('communes of the ain department', 'departments of france'), 0.057969869439630416), (('ain', 'communes of the ain department'), 0.05786664097179347), (('communes of the vaucluse department', 'france'), 0.05769413809053141), (('communes of the calvados department', 'france'), 0.05763990496368279), (('communes of the calvados department', 'departments of france'), 0.057525982373471694), (('communes of the ard\\u00e8che department', 'departments of france'), 0.05622435402990622), (('communes of the ard\\u00e8che department', 'departments of france'), 0.056224354029906214), (('arkansas', 'list of cities in arkansas'), 0.05553318554754128), (('idaho', 'list of cities in idaho'), 0.05552855951510441), (('list of cities in idaho', 'united states'), 0.05539981894736573), (('communes of the alpes-maritimes department', 'france'), 0.05537579260779012), (('list of cities in arkansas', 'united states'), 0.05535927250406302), (('city', 'list of cities in arkansas'), 0.05466340875710438), (('communes of the alpes-maritimes department', 'departments of france'), 0.05464413681987893), (('city', 'list of cities in idaho'), 0.05444266001148339), (('essex', 'list of civil parishes in essex'), 0.053060339890832905), (('kentucky', 'list of cities in kentucky'), 0.05182799953150506), (('belgium', 'list of cities in belgium'), 0.051657947794183354), (('list of cities in kentucky', 'united states'), 0.05153528339808095), (('city', 'list of cities in kentucky'), 0.05153495768746181), (('arrondissement of largenti\\u00e8re', 'france'), 0.05073709613649795), (('communes of the bouches-du-rh\\u00f4ne department', 'france'), 0.050382819755594985), (('romania', 'suceava county'), 0.049781900828279074), (('arrondissement of bayeux', 'france'), 0.049682358935990927), (('arrondissement of largenti\\u00e8re', 'departments of france'), 0.04961734228005818), (('arrondissement of bayeux', 'departments of france'), 0.04922972859556402), (('belgium', 'flemish brabant'), 0.04915669612643677), (('arrondissement of privas', 'france'), 0.04894222203599536), (('communes of the bouches-du-rh\\u00f4ne department', 'departments of france'), 0.0488949842822584), (('arrondissement of belley', 'france'), 0.048547418874361), (('belgium', 'hainaut (province)'), 0.0484770605606388), (('arrondissement of belley', 'departments of france'), 0.04772628836265976), (('japan', 'list of japan international footballers'), 0.0467712707691298), (('arrondissement of privas', 'departments of france'), 0.04607602212495078), (('belgium', 'telephone numbers in belgium'), 0.045023654710355465), (('list of towns in virginia', 'united states'), 0.044572460127338676), (('japan', 'list of japanese footballers'), 0.044162435449358384), (('florida', 'list of settlements in florida'), 0.04412688848396136), (('belgium', 'west flanders'), 0.04333382305659213), (('belgium', 'li\\u00e8ge (province)'), 0.042843589879673336), (('france', 'list of french departments by population'), 0.042772662573012106), (('antwerp (province)', 'belgium'), 0.04252577877688411), (('departments of france', 'list of french departments by population'), 0.03893392592316101), (('belgium', 'east flanders'), 0.03860339181807095), (('belgium', 'luxembourg (belgium)'), 0.03852015151158596), (('belgium', 'namur (province)'), 0.03823283481967761), (('communes of the aube department', 'departments of france'), 0.03702625547753828), (('communes of the aube department', 'departments of france'), 0.037026255477538274), (('communes of the aube department', 'france'), 0.03692045600855769), (('list of settlements in florida', 'united states'), 0.03589471350268824), (('belgium', 'limburg (belgium)'), 0.03575355189930803), (('idaho', 'united states'), 0.0355952687447257), (('france', 'list of cities in france'), 0.03467150578983111), (('departments of france', 'france'), 0.034202524418152476), (('city', 'list of settlements in florida'), 0.03412649689115581), (('arrondissement of privas', 'communes of the ard\\u00e8che department'), 0.032148621579164934), (('arrondissement of largenti\\u00e8re', 'communes of the ard\\u00e8che department'), 0.03202904340084403), (('italy', 'list of cities in italy'), 0.03142299657472391), (('germany', 'list of cities in germany with more than 100,000 inhabitants'), 0.03024578804990069), (('1999', 'communes of the vaucluse department'), 0.030169316658287204), (('chicago', 'mayor of chicago'), 0.02985943456704292), (('1999', 'communes of the mayenne department'), 0.02954448927585469), (('germany', 'north rhine-westphalia'), 0.027397240632923807), (('germany', 'urban districts of germany'), 0.026757625398486154), (('mayor of chicago', 'united states'), 0.026026886831909894), (('italy', 'regions of italy'), 0.025849422866570482), (('2006', 'list of french departments by population'), 0.025117418378834803), (('kentucky', 'united states'), 0.025075577757247232), (('list of ioc country codes', 'summer olympic games'), 0.024115226450299853), (('list of tallest buildings in the united states', 'united states'), 0.024013933488072663), (('france', 'lists of cantons of france'), 0.02356991173497723), (('italy', 'provinces of italy'), 0.023345087270139692), (('british rail', 'list of british rail modern traction locomotive classes'), 0.02332854018752648), (('germany', 'saxony'), 0.0228601423754174), (('france', 'lists of communes of france'), 0.022692780316570047), (('segunda divisi\\u00f3n', 'spain'), 0.02257759773050486), (('japan', 'list of j. league players'), 0.022093835189117107), (('departments of france', 'lists of cantons of france'), 0.02198406494134653), (('japan', 'list of provinces of japan'), 0.02123295650033441), (('provinces of italy', 'regions of italy'), 0.021068256192122024), (('japan', 'list of yokohama f. marinos players'), 0.02081878738775042), (('japan', 'prefectures of japan'), 0.020596698796665633), (('british rail', 'list of british rail diesel multiple unit classes'), 0.020319967541153536), (('alaska', 'list of cities in alaska'), 0.02024830581106988), (('departments of france', 'lists of communes of france'), 0.02018757569860292), (('departments of france', 'lists of communes of france'), 0.02018757569860292), (('belgium', 'list of rivers of belgium'), 0.019806210571943747), (('japan', 'list of sanfrecce hiroshima players'), 0.019733975730893114), (('list of united states cities by population', 'united states'), 0.019693456614132902), (('list of ioc country codes', 'winter olympic games'), 0.01909340727062727), (('france', 'french civil aviation university'), 0.01875333688327837), (('2011\\u201312 juventus f.c. season', 'italy'), 0.01838422721850844), (('japan', 'list of japanese football teams'), 0.01810377378063885), (('italy', 'serie d'), 0.017935797129511377), (('administrative divisions of azerbaijan', 'azerbaijan'), 0.01785993143969559), (('germany', 'zwickau rural district'), 0.01735666177041608), (('ichinomiya', 'list of provinces of japan'), 0.017143720078395125), (('india', 'uttar pradesh'), 0.01685745703154031), (('list of cities in italy', 'regions of italy'), 0.016731541709562366), (("list of u.s. states' largest cities", 'united states'), 0.016672455272457105), (("list of u.s. states' largest cities", 'united states'), 0.016672455272457105), (('echl', 'national hockey league'), 0.01657206572606903), (('comparison of regions of japan', 'japan'), 0.016537851818537046), (('australia', 'new south wales'), 0.016477189883082482), (('american hockey league', 'echl'), 0.01640276835507372), (('core cities of japan', 'japan'), 0.01631532547343719), (('germany', 'list of administrative divisions of germany'), 0.016226527348799185), (('2007-08 nhl season', 'national hockey league'), 0.01592716797143552), (('departments of france', 'list of cities in france'), 0.01586932159855949), (('germany', 'germany national football team'), 0.015757657564001014), (('japan', 'list of the 53 stations of the t\\u014dkaid\\u014d'), 0.01575185688102915), (('british rail', 'list of british rail electric multiple unit classes'), 0.015734483269504558), (('list of cities in italy', 'provinces of italy'), 0.015659268377260872), (('city', 'list of united states cities by population'), 0.015406911810653432), (('docg', 'italy'), 0.015154047071346266), (('list of ioc country codes', 'olympic games'), 0.01502037056783245), (('association of american universities', 'united states'), 0.014953804116004736), (('australia', 'list of cities in australia'), 0.014855728616322485), (('list of u.s. states', 'united states'), 0.01475006903493294), (('2009\\u201310 nhl season', 'national hockey league'), 0.014660947055393071), (('france', 'list of bastides'), 0.014618597421911461), (('list of u.s. state capitals', 'united states'), 0.01443583158277038), (('city', 'list of cities in alaska'), 0.01429093196706684), (('germany', 'hesse'), 0.014284916160248663), (('friends', 'united states'), 0.014226047814786277), (('england', 'list of english football stadiums by capacity'), 0.014083960626477198), (('australia', 'sydney'), 0.013989488275402051)]


In [785]:
answer[0]

(('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'),
 0.06825002019931593)

In [786]:
answer2 =  [(('communes of the yonne department', 'france'), 0.06851956783485441), (('communes of the yonne department', 'departments of france'), 0.0684190413603516), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'), 0.06838308170477443), (('communes of the yonne department', 'yonne'), 0.06830510902166947), (('communes of france', 'communes of the yonne department'), 0.06830108901309988), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'departments of france'), 0.06829913140425876), (('illinois', 'list of cities in illinois'), 0.06825788908552215), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'pyr\\u00e9n\\u00e9es-atlantiques'), 0.06824276476154595), (('commune in france', 'communes of the pyr\\u00e9n\\u00e9es-atlantiques department'), 0.06819610556035147), (('communes of the allier department', 'france'), 0.06763361651363364)]


In [747]:
answer2[0][0]

('communes of the yonne department', 'france')

In [766]:
answer2

[(('communes of the yonne department', 'france'), 0.06851956783485441),
 (('communes of the yonne department', 'departments of france'),
  0.0684190413603516),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'),
  0.06838308170477443),
 (('communes of the yonne department', 'yonne'), 0.06830510902166947),
 (('communes of france', 'communes of the yonne department'),
  0.06830108901309988),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department',
   'departments of france'),
  0.06829913140425876),
 (('illinois', 'list of cities in illinois'), 0.06825788908552215),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department',
   'pyr\\u00e9n\\u00e9es-atlantiques'),
  0.06824276476154595),
 (('commune in france',
   'communes of the pyr\\u00e9n\\u00e9es-atlantiques department'),
  0.06819610556035147),
 (('communes of the allier department', 'france'), 0.06763361651363364)]

In [788]:
for item1 in answer2 :
    pair1 = item1[0]
    for item2 in answer:
        pair2=item2[0]
        if pair2==pair1:
            item2 = item1
    if item1 not in answer:
        answer.append(item1)
        
        
        
    

In [789]:
len(answer)

165

In [790]:
new_answer = sorted(answer, key=lambda x: x[1], reverse=True)

In [791]:
grader = new_answer[:100]

In [792]:
len(grader)

100

In [793]:
grader

[(('communes of the yonne department', 'france'), 0.06851956783485441),
 (('communes of the yonne department', 'departments of france'),
  0.0684190413603516),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'),
  0.06838308170477443),
 (('communes of the yonne department', 'yonne'), 0.06830510902166947),
 (('communes of france', 'communes of the yonne department'),
  0.06830108901309988),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department',
   'departments of france'),
  0.06829913140425876),
 (('illinois', 'list of cities in illinois'), 0.06825788908552215),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'),
  0.06825002019931593),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department',
   'pyr\\u00e9n\\u00e9es-atlantiques'),
  0.06824276476154595),
 (('commune in france',
   'communes of the pyr\\u00e9n\\u00e9es-atlantiques department'),
  0.06819610556035147),
 (('communes of the pyr\\u00e9n\\u00e9es-atlantiq